In [1]:
pip install -U imbalanced-learn

Note: you may need to restart the kernel to use updated packages.


In [2]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import mean_squared_error
from sklearn.ensemble import RandomForestRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.linear_model import LinearRegression, Lasso
from sklearn.preprocessing import LabelEncoder, StandardScaler

In [8]:
bd_train = pd.read_csv(r"C:\Users\akans\Downloads\Machine Learning (EDVANCER IITK)\PROJECT 3 - PHARMA (Public Saftey)\counterfeit_train.csv")
bd_test = pd.read_csv("counterfeit_test.csv")

In [9]:
bd_train.head(5)

,Medicine_ID,Counterfeit_Weight,DistArea_ID,Active_Since,Medicine_MRP,Medicine_Type,SidEffect_Level,Availability_rating,Area_Type,Area_City_Type,Area_dist_level,Counterfeit_Sales
0,RRA15,13.100,Area046,1995,160.2366,Antimalarial,critical,0.070422,DownTown,Tier 1,Small,1775.5026
1,YVV26,NaN,Area027,1983,110.4384,Mstablizers,mild,0.013000,CityLimits,Tier 3,Medium,3069.1520
2,LJC15,9.025,Area046,1995,259.4092,Cardiac,mild,0.060783,DownTown,Tier 1,Small,2603.0920
3,GWC40,11.800,Area046,1995,99.9830,OralContraceptives,mild,0.065555,DownTown,Tier 1,Small,1101.7130
4,QMN13,NaN,Area019,1983,56.4402,Hreplacements,critical,0.248859,MidTownResidential,Tier 1,Small,158.9402


In [10]:
bd_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6818 entries, 0 to 6817
Data columns (total 12 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   Medicine_ID          6818 non-null   object 
 1   Counterfeit_Weight   5652 non-null   float64
 2   DistArea_ID          6818 non-null   object 
 3   Active_Since         6818 non-null   int64  
 4   Medicine_MRP         6818 non-null   float64
 5   Medicine_Type        6818 non-null   object 
 6   SidEffect_Level      6818 non-null   object 
 7   Availability_rating  6818 non-null   float64
 8   Area_Type            6818 non-null   object 
 9   Area_City_Type       6818 non-null   object 
 10  Area_dist_level      6818 non-null   object 
 11  Counterfeit_Sales    6818 non-null   float64
dtypes: float64(4), int64(1), object(7)
memory usage: 639.3+ KB


In [11]:
bd_test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1705 entries, 0 to 1704
Data columns (total 11 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   Medicine_ID          1705 non-null   object 
 1   Counterfeit_Weight   1408 non-null   float64
 2   DistArea_ID          1705 non-null   object 
 3   Active_Since         1705 non-null   int64  
 4   Medicine_MRP         1705 non-null   float64
 5   Medicine_Type        1705 non-null   object 
 6   SidEffect_Level      1705 non-null   object 
 7   Availability_rating  1705 non-null   float64
 8   Area_Type            1705 non-null   object 
 9   Area_City_Type       1705 non-null   object 
 10  Area_dist_level      1705 non-null   object 
dtypes: float64(3), int64(1), object(7)
memory usage: 146.7+ KB


In [12]:
X = bd_train.drop(columns=['Counterfeit_Sales', 'Medicine_ID'])  # Exclude Medicine_ID
y = bd_train['Counterfeit_Sales']

In [13]:
for col in X.select_dtypes(include=['float64', 'int64']).columns:
    X[col].fillna(X[col].median(), inplace=True)

In [14]:
label_encoders = {}
for col in X.select_dtypes(include=['object']).columns:
    le = LabelEncoder()
    X[col] = le.fit_transform(X[col])
    label_encoders[col] = le

In [15]:
for col in bd_test.select_dtypes(include=['float64', 'int64']).columns:
    bd_test[col].fillna(bd_test[col].median(), inplace=True)

In [16]:
for col in bd_test.select_dtypes(include=['object']).columns:
    if col in label_encoders:
        bd_test[col] = label_encoders[col].transform(bd_test[col])

In [17]:
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

In [18]:
def train_and_evaluate_model(model, model_name):
    """
    Train the model, evaluate it on the validation set, and predict test values.
    
    Parameters:
    model: A scikit-learn estimator.
    model_name: Name of the model for logging purposes.
    
    Returns:
    test_predictions: Predicted values for the test set.
    """
    # Fit the model
    print(f"Training {model_name}...")  # Proper indentation
    model.fit(X_train, y_train)
    
    # Predict on validation data
    y_val_pred = model.predict(X_val)
    
    # Calculate RMSE
    rmse = np.sqrt(mean_squared_error(y_val, y_val_pred))
    print(f"{model_name} Validation RMSE: {rmse}")
    
    # Predict on test data
    test_features = bd_test.drop(columns=['Medicine_ID'])
    test_predictions = model.predict(test_features)
    
    return test_predictions

In [19]:
scaler = StandardScaler()

In [20]:
X_train_scaled = scaler.fit_transform(X_train)
X_val_scaled = scaler.transform(X_val)
bd_test_scaled = scaler.transform(bd_test.drop(columns=['Medicine_ID']))

In [21]:
lr_model = LinearRegression()

In [22]:
lr_model.fit(X_train_scaled, y_train)

LinearRegression()

In [23]:
lr_val_pred = lr_model.predict(X_val_scaled)

In [24]:
lr_rmse = np.sqrt(mean_squared_error(y_val, lr_val_pred))


In [25]:
print(f"Linear Regression Validation RMSE: {lr_rmse}")

Linear Regression Validation RMSE: 1134.9547614718535


In [26]:
lr_test_predictions = lr_model.predict(bd_test_scaled)

In [27]:
dt_param_grid = {
    'max_depth': [5, 10, 20],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4]
}

In [28]:
dt_model = DecisionTreeRegressor(random_state=42)
dt_grid_search = GridSearchCV(dt_model, dt_param_grid, cv=3, scoring='neg_mean_squared_error', verbose=1)

In [29]:
param_grid = {
    'n_estimators': [50, 100, 200],
    'max_depth': [None, 10, 20],
    'min_samples_split': [2, 5, 10]
}

In [30]:
dt_grid_search.fit(X_train, y_train)

Fitting 3 folds for each of 27 candidates, totalling 81 fits


GridSearchCV(cv=3, estimator=DecisionTreeRegressor(random_state=42),
             param_grid={'max_depth': [5, 10, 20],
                         'min_samples_leaf': [1, 2, 4],
                         'min_samples_split': [2, 5, 10]},
             scoring='neg_mean_squared_error', verbose=1)

In [31]:
best_dt_model = dt_grid_search.best_estimator_

In [32]:
dt_val_pred = best_dt_model.predict(X_val)

In [33]:
dt_rmse = np.sqrt(mean_squared_error(y_val, dt_val_pred))
print(f"Decision Tree Validation RMSE: {dt_rmse}")

Decision Tree Validation RMSE: 1077.6061127674707


In [34]:
dt_test_predictions = best_dt_model.predict(bd_test.drop(columns=['Medicine_ID']))

In [35]:
rf_param_grid = {
    'n_estimators': [50, 100, 200],
    'max_depth': [10, 20, None],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4],
    'max_features': ['sqrt', 'log2']
}

In [36]:
rf_model = RandomForestRegressor(random_state=42)

In [37]:
rf_grid_search = GridSearchCV(rf_model, rf_param_grid, cv=3, scoring='neg_mean_squared_error', verbose=1)

In [38]:
rf_grid_search.fit(X_train, y_train)

Fitting 3 folds for each of 162 candidates, totalling 486 fits


GridSearchCV(cv=3, estimator=RandomForestRegressor(random_state=42),
             param_grid={'max_depth': [10, 20, None],
                         'max_features': ['sqrt', 'log2'],
                         'min_samples_leaf': [1, 2, 4],
                         'min_samples_split': [2, 5, 10],
                         'n_estimators': [50, 100, 200]},
             scoring='neg_mean_squared_error', verbose=1)

In [39]:
best_rf_model = rf_grid_search.best_estimator_

In [40]:
rf_val_pred = best_rf_model.predict(X_val)

In [41]:
rf_rmse = np.sqrt(mean_squared_error(y_val, rf_val_pred))

In [42]:
print(f"Random Forest Validation RMSE: {rf_rmse}")
rf_test_predictions = best_rf_model.predict(bd_test.drop(columns=['Medicine_ID']))

Random Forest Validation RMSE: 1077.7967527231208


In [43]:
# Linear Regression Submission
lr_submission = pd.DataFrame({'Counterfeit_Sales': lr_test_predictions})
lr_submission.to_csv('submission_linear_regression.csv', index=False)

In [44]:
# Decision Tree Regressor Submission
dt_submission = pd.DataFrame({'Counterfeit_Sales': dt_test_predictions})
dt_submission.to_csv('submission_decision_tree.csv', index=False)

In [45]:
# Random Forest Regressor Submission
rf_submission = pd.DataFrame({'Counterfeit_Sales': rf_test_predictions})
rf_submission.to_csv('submission_random_forest.csv', index=False)

In [ ]:
#A lower validation RMSE (Root Mean Squared Error) is better because 
# RMSE measures the error between your model’s predictions and the actual values.

# RMSE represents the standard deviation of prediction errors 
# (how far predictions are from actual values).

# A higher RMSE means your model’s predictions are less accurate (more error).

# A lower RMSE means your model’s predictions are closer to actual values (less error).
# Try to minimize RMSE to improve model performance. 